In [ ]:
import pandas as pd

# 新段落

In [ ]:
url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv"
df = pd.read_csv(url,index_col=0)
df.head()

,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,...,12/5/22,12/6/22,12/7/22,12/8/22,12/9/22,12/10/22,12/11/22,12/12/22,12/13/22,12/14/22
UID,,,,,,,,,,,,,,,,,,,,,
84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,"Autauga, Alabama, US",...,230,230,230,230,230,230,230,230,230,230
84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,"Baldwin, Alabama, US",...,716,716,717,717,717,717,717,717,717,717
84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,"Barbour, Alabama, US",...,103,103,103,103,103,103,103,103,103,103
84001007,US,USA,840,1007.0,Bibb,Alabama,US,32.996421,-87.125115,"Bibb, Alabama, US",...,108,108,108,108,108,108,108,108,108,108
84001009,US,USA,840,1009.0,Blount,Alabama,US,33.982109,-86.567906,"Blount, Alabama, US",...,259,259,260,260,260,260,260,260,260,260


In [ ]:
death_by_state = df.groupby('Province_State')[df.columns[11:]].sum()
death_by_state.head(10)

,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,...,12/5/22,12/6/22,12/7/22,12/8/22,12/9/22,12/10/22,12/11/22,12/12/22,12/13/22,12/14/22
Province_State,,,,,,,,,,,,,,,,,,,,,
Alabama,0,0,0,0,0,0,0,0,0,0,...,20652,20652,20682,20682,20682,20682,20682,20682,20682,20704
Alaska,0,0,0,0,0,0,0,0,0,0,...,1436,1436,1436,1436,1436,1436,1436,1436,1436,1436
American Samoa,0,0,0,0,0,0,0,0,0,0,...,34,34,34,34,34,34,34,34,34,34
Arizona,0,0,0,0,0,0,0,0,0,0,...,31751,31751,31822,31822,31822,31822,31822,31822,31822,31929
Arkansas,0,0,0,0,0,0,0,0,0,0,...,12585,12594,12603,12613,12623,12631,12631,12631,12638,12647
California,0,0,0,0,0,0,0,0,0,0,...,97645,97655,97682,97817,97834,97834,97834,97873,97892,97918
Colorado,0,0,0,0,0,0,0,0,0,0,...,13652,13668,13682,13697,13704,13704,13704,13715,13740,13743
Connecticut,0,0,0,0,0,0,0,0,0,0,...,11587,11602,11603,11603,11603,11603,11603,11603,11656,11659
Delaware,0,0,0,0,0,0,0,0,0,0,...,3172,3172,3172,3172,3184,3184,3184,3184,3184,3185


In [ ]:
test = pd.DataFrame(death_by_state.iloc[1])
current_state = test.columns[0]
current_state
##### WARNING: is the data already counting cumulative or is it daily cases?
test.columns=['cum_death']
test['new_death']=test['cum_death'].diff().fillna(test['cum_death'])
#print(df)
#test.reset_index()
#test.head()

In [ ]:
from math import exp
def model(params, time_step):
  p,q,m = params
  N=[]
  for t in range(1,time_step):
    current=m*(1-exp(-(p+q)*t))/(1+(q/p)*exp(-(p+q)*t))
    last=m*(1-exp(-(p+q)*(t-1)))/(1+(q/p)*exp(-(p+q)*(t-1)))
    N.append(current-last)
  return N


def error(params, time_step, actual_N):
  pred_N = model(params,time_step)
  N_error = []
  for t in range(0,time_step-1):
    N_error.append((actual_N[t+1]-pred_N[t])**2)
  total_error = sum(N_error)
  return total_error




In [ ]:
import numpy as np
from scipy.optimize import minimize
import pandas as pd

In [ ]:
initial_value = [0.027,0.003,0.03]
bnds = [(0, None), (0, None), (0, None)]
res = minimize(error,initial_value, args = (1028,test.new_death), bounds = bnds, tol=1e-8, method = 'Powell')

In [ ]:
print(res)

   direc: array([[-1.20012006e-05,  7.47837393e-04,  9.60104503e+01],
       [-3.04291749e-05,  1.12323411e-03, -1.39193406e+02],
       [-4.78326155e-06,  5.60700835e-04, -6.38933555e+01]])
     fun: 28422.787858961932
 message: 'Optimization terminated successfully.'
    nfev: 1198
     nit: 16
  status: 0
 success: True
       x: array([1.08686633e-04, 6.25513198e-03, 1.75989179e+03])


In [ ]:
def prediction(params,current_time, time_range):
  p,q,m = params
  N_t = []
  for t in range(current_time,current_time+time_range):
    current=m*(1-exp(-(p+q)*t))/(1+(q/p)*exp(-(p+q)*t))
    last=m*(1-exp(-(p+q)*(t-1)))/(1+(q/p)*exp(-(p+q)*(t-1)))
    N_t.append(current-last)
  return N_t




In [ ]:
len(death_by_state)

58

In [ ]:
final_prediction_df = pd.DataFrame()
for state in range(0,58):
  test = pd.DataFrame(death_by_state.iloc[state])
  current_state = test.columns[0]
  ##### WARNING: is the data already counting cumulative or is it daily cases?
  test.columns=['cum_death']
  test['new_death']=test['cum_death'].diff().fillna(test['cum_death'])
  initial_value = [0.027,0.03,100]
  bnds = [(0, None), (0, None), (10, None)]
  res = minimize(error,initial_value, args = (1028,test.new_death), bounds = bnds, tol=1e-8, method = 'Powell')
  print(res.x)
  final_prediction_df[current_state] = prediction(res.x,1018,400)


[6.85348468e-05 7.45512874e-02 7.15048376e+02]
[1.95890521e-06 1.32754303e-02 1.19923394e+03]
[4.74359040e-05 5.64467686e-03 6.25494210e+01]
[4.93288970e-04 4.34575555e-03 3.43540333e+04]
[1.73888509e-04 1.01622182e-01 4.71895508e+01]
[4.80542406e-06 2.29601548e-02 6.39887423e+04]
[2.93835533e-03 2.68151079e-02 2.15148859e+03]
[2.12348097e-03 2.85751395e-02 5.42844123e+03]
[1.71509248e-03 2.69908748e-02 7.61501051e+02]
[1.51278758e-17 5.02591791e-09 1.00000000e+01]
[1.68958999e-03 2.78090743e-02 7.32408180e+02]
[3.35690801e-04 4.69983359e-03 9.05015233e+04]
[1.42838592e-07 1.85237122e-02 2.36354418e+03]
[3.96790131e-04 1.58056185e-04 1.00000001e+01]
[4.40574432e-05 2.41494897e-02 1.48842242e+02]
[9.3192310e-05 6.1712102e-03 1.8597482e+03]
[1.92092126e-06 1.32804060e-01 7.47200918e+01]
[7.80890927e-04 3.61002049e-03 4.25513219e+04]
[4.18929984e-03 1.95916011e-10 1.03148714e+04]
[1.23673804e-05 2.25354100e-02 5.94767682e+03]
[1.87986134e-05 2.06375733e-02 5.80223146e+03]
[1.23394774e-05 

In [ ]:
final_prediction_df

,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
0,0.0,0.143865,0.070176,10.473993,0.0,0.000499,4.592948e-11,6.457412e-11,7.548806e-11,1.512795e-16,...,31.233598,19.485715,0.0,0.0,0.039289,11.293176,0.0,9.342665,0.000526,0.000014
1,0.0,0.142002,0.069990,10.429493,0.0,0.000488,4.501999e-11,6.366463e-11,7.332801e-11,1.512795e-16,...,31.241359,19.380377,0.0,0.0,0.039044,11.253100,0.0,9.347118,0.000517,0.000014
2,0.0,0.140162,0.069803,10.385155,0.0,0.000477,4.320100e-11,6.275513e-11,7.150902e-11,1.512795e-16,...,31.249121,19.275567,0.0,0.0,0.038800,11.213126,0.0,9.351573,0.000508,0.000014
3,0.0,0.138346,0.069615,10.340978,0.0,0.000466,4.229150e-11,5.911716e-11,6.923528e-11,1.512795e-16,...,31.256885,19.171283,0.0,0.0,0.038558,11.173252,0.0,9.356030,0.000498,0.000014
4,0.0,0.136553,0.069427,10.296961,0.0,0.000456,4.138201e-11,5.820766e-11,6.730261e-11,1.512795e-16,...,31.264650,19.067523,0.0,0.0,0.038317,11.133481,0.0,9.360488,0.000489,0.000013
5,0.0,0.134783,0.069239,10.253106,0.0,0.000445,3.956302e-11,5.547918e-11,6.548362e-11,1.512795e-16,...,31.272416,18.964284,0.0,0.0,0.038077,11.093810,0.0,9.364949,0.000481,0.000013
6,0.0,0.133035,0.069050,10.209411,0.0,0.000435,3.819878e-11,5.456968e-11,6.366463e-11,1.512795e-16,...,31.280183,18.861564,0.0,0.0,0.037839,11.054242,0.0,9.369411,0.000472,0.000013
7,0.0,0.131309,0.068861,10.165876,0.0,0.000425,3.774403e-11,5.275069e-11,6.173195e-11,1.512795e-16,...,31.287952,18.759361,0.0,0.0,0.037602,11.014775,0.0,9.373875,0.000464,0.000012
8,0.0,0.129606,0.068672,10.122500,0.0,0.000416,3.592504e-11,5.184120e-11,6.002665e-11,1.512795e-16,...,31.295722,18.657673,0.0,0.0,0.037366,10.975410,0.0,9.378341,0.000455,0.000012
9,0.0,0.127924,0.068482,10.079284,0.0,0.000406,3.547029e-11,5.002221e-11,5.843503e-11,1.512795e-16,...,31.303494,18.556498,0.0,0.0,0.037132,10.936147,0.0,9.382809,0.000447,0.000012


In [ ]:
final_prediction_df.sum(axis = 1)

0     276.415553
1     275.252001
2     274.092548
3     272.937188
4     271.785914
5     270.638720
6     269.495599
7     268.356543
8     267.221547
9     266.090604
10    264.963706
11    263.840846
12    262.722018
13    261.607214
14    260.496427
15    259.389649
16    258.286874
17    257.188094
18    256.093301
19    255.002488
20    253.915648
21    252.832772
22    251.753853
23    250.678883
24    249.607854
25    248.540759
26    247.477589
27    246.418336
28    245.362993
29    244.311552
dtype: float64

In [ ]:
weekly_df = death_by_state[death_by_state.columns[::7]]
weekly_df.head()

,1/22/20,1/29/20,2/5/20,2/12/20,2/19/20,2/26/20,3/4/20,3/11/20,3/18/20,3/25/20,...,9/28/22,10/5/22,10/12/22,10/19/22,10/26/22,11/2/22,11/9/22,11/16/22,11/23/22,11/30/22
Province_State,,,,,,,,,,,,,,,,,,,,,
Alabama,0,0,0,0,0,0,0,0,0,1,...,20395,20422,20473,20505,20533,20558,20558,20617,20631,20652
Alaska,0,0,0,0,0,0,0,0,0,1,...,1364,1393,1393,1393,1413,1413,1413,1436,1436,1436
American Samoa,0,0,0,0,0,0,0,0,0,0,...,34,34,34,34,34,34,34,34,34,34
Arizona,0,0,0,0,0,0,0,0,0,6,...,31370,31406,31455,31514,31548,31573,31613,31647,31709,31751
Arkansas,0,0,0,0,0,0,0,0,0,2,...,12131,12235,12334,12396,12452,12506,12523,12528,12545,12564


In [ ]:
weekly_final_prediction_df = pd.DataFrame()
for state in range(0,58):
  test = pd.DataFrame(weekly_df.iloc[state])
  current_state = test.columns[0]
  ##### WARNING: is the data already counting cumulative or is it daily cases?
  test.columns=['cum_death']
  test['new_death']=test['cum_death'].diff().fillna(test['cum_death'])
  initial_value = [0.027,0.03,500]
  bnds = [(0, None), (0, None), (0, None)]
  res = minimize(error,initial_value, args = (149,test.new_death), bounds = bnds, tol=1e-8, method = 'Powell')
  print(res.x)
  weekly_final_prediction_df[current_state] = prediction(res.x,147,4)

[2.53662352e-03 3.92312856e-02 2.17136281e+04]
[4.52641064e-04 5.09960391e-02 1.47285756e+03]
[3.57395004e-03 1.33183661e-02 4.48146283e+01]
[3.43372743e-03 3.05269579e-02 3.43702057e+04]
[2.66275774e-03 3.12050306e-02 1.38800931e+04]
[2.06263090e-04 1.21178910e-01 7.24063712e+04]
[3.20626451e-02 1.16183360e-10 5.27822425e+03]
[2.92646453e-02 2.81844403e+00 3.34078602e-05]
[3.12357184e-02 4.30241101e-09 7.90453323e+02]
[3.53125138e-09 5.56901951e-09 6.43449008e-09]
[9.41167042e-03 2.09157117e-01 8.70870944e+02]
[2.35516153e-03 3.26506380e-02 9.07310363e+04]
[2.85444702e-03 3.14652795e-02 4.47340466e+04]
[0.01051798 0.3051801  3.41965871]
[7.72908374e-06 3.20634994e+00 7.64195563e-02]
[6.46040022e-04 4.31410170e-02 1.86911744e+03]
[1.24122522e-03 4.18165501e-02 5.65449680e+03]
[5.46105007e-03 2.52364494e-02 4.25895024e+04]
[3.69622168e-03 2.76973013e-02 2.75777080e+04]
[3.72143107e-03 3.43623597e-02 1.04236208e+04]
[3.01691855e-03 3.31493823e-02 1.02760462e+04]
[1.23231917e-03 3.8553935

In [ ]:
weekly_final_prediction_df

,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
0,30.737555,4.084786,0.174536,70.778633,35.755428,0.097930,1.543645,0.0,0.254214,2.272181e-17,...,98.522272,130.273301,14.356599,0.0,0.219721,76.765895,67.110227,25.131085,48.393448,4.685878
1,29.559719,3.902083,0.172993,68.680416,34.739994,0.086736,1.494937,0.0,0.246396,2.272181e-17,...,95.742944,125.396130,13.899406,0.0,0.216256,74.857779,65.568653,24.242326,47.152261,4.490882
2,28.423980,3.726545,0.171447,66.636268,33.748125,0.076822,1.447766,0.0,0.238819,2.272181e-17,...,93.023694,120.689757,13.454407,0.0,0.212838,72.984703,64.047737,23.379393,45.935887,4.303139
3,27.329070,3.557984,0.169897,64.645307,32.779600,0.068041,1.402083,0.0,0.231474,2.272181e-17,...,90.364377,116.149098,13.021437,0.0,0.209467,71.146721,62.548029,22.541964,44.744230,4.122453


In [ ]:
weekly_final_prediction_df.sum(axis=1)

0    1966.177380
1    1908.668365
2    1852.592750
3    1797.931651
dtype: float64

In [ ]:
monthly_df = death_by_state[death_by_state.columns[::30]]
monthly_df.head()

,1/22/20,2/21/20,3/22/20,4/21/20,5/21/20,6/20/20,7/20/20,8/19/20,9/18/20,10/18/20,...,2/10/22,3/12/22,4/11/22,5/11/22,6/10/22,7/10/22,8/9/22,9/8/22,10/8/22,11/7/22
Province_State,,,,,,,,,,,,,,,,,,,,,
Alabama,0,0,0,183,529,838,1291,1944,2428,2788,...,17502,18832,19379,19606,19692,19786,19974,20239,20473,20558
Alaska,0,0,0,9,10,12,18,29,45,67,...,1114,1201,1228,1268,1285,1285,1309,1338,1393,1413
American Samoa,0,0,0,0,0,0,0,0,0,0,...,0,0,11,29,31,31,33,34,34,34
Arizona,0,0,2,208,764,1346,2784,4634,5451,5827,...,26856,28090,29681,30230,30372,30572,30842,31162,31406,31573
Arkansas,0,0,0,42,110,224,363,631,1173,1704,...,9937,10892,11313,11410,11512,11608,11775,11951,12301,12521


In [ ]:
monthly_final_prediction_df = pd.DataFrame()
for state in range(0,58):
  test = pd.DataFrame(monthly_df.iloc[state])
  current_state = test.columns[0]
  ##### WARNING: is the data already counting cumulative or is it daily cases?
  test.columns=['cum_death']
  test['new_death']=test['cum_death'].diff().fillna(test['cum_death'])
  initial_value = [0.027,0.03,500]
  bnds = [(0, None), (0, None), (0, None)]
  res = minimize(error,initial_value, args = (34,test.new_death), bounds = bnds, tol=1e-8, method = 'Powell')
  print(res.x)
  monthly_final_prediction_df[current_state] = prediction(res.x,33,2)

[1.09724254e-02 1.66298404e-01 2.17996097e+04]
[ 0.02684564  1.71448828 14.37953792]
[1.19940817e-03 1.68337786e-01 7.01835424e+01]
[1.48915765e-02 1.27672369e-01 3.46636125e+04]
[1.14451831e-02 1.34034738e-01 1.38623322e+04]
[5.35237879e-02 1.04530323e+00 7.39875900e+03]
[1.20972417e-01 9.13257263e-10 6.07894933e+03]
[4.10707613e-02 1.01075066e+00 6.31964760e+03]
[1.34604120e-08 2.12809506e+01 1.14047808e+02]
[3.26854551e-09 4.30765061e-09 2.02775460e-19]
[1.07007376e-07 1.89569394e+01 1.02737586e+02]
[1.01454213e-02 1.39112837e-01 9.12009821e+04]
[1.22557219e-02 1.34034755e-01 4.48517831e+04]
[6.91080988e-05 5.25268658e-09 1.32095586e+03]
[6.12289269e-03 1.39868027e-01 4.77216164e+02]
[3.20142015e-03 1.74532836e-01 1.92992348e+03]
[5.46213323e-03 1.76437466e-01 5.69701164e+03]
[2.34913681e-02 1.07135084e-01 4.26821792e+04]
[1.58923368e-02 1.17443116e-01 2.76827646e+04]
[1.62962386e-02 1.45902717e-01 1.05235482e+04]
[1.30010451e-02 1.38932003e-01 1.03802358e+04]
[5.20900843e-03 1.6570

In [ ]:
monthly_final_prediction_df.sum(axis=1)

0    10065.541882
1     8916.678214
dtype: float64

In [ ]:
monthly_final_prediction_df.to_csv('monthly.csv')

In [ ]:
weekly_final_prediction_df.to_csv('weekly.csv')

In [ ]:
final_prediction_df.to_csv('daily.csv')